### Load libraries

In [1]:
import time
import hmac
import hashlib
import requests
import json
import pandas as pd

/home/ngtran/api authorization/api_call_venv/lib/python3.12/site-packages/numpy/_core/getlimits.py:548: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


### First, we need acquired access token to authenticate API calls
* `partner_id`, `partner_key` can be attained from your APP in your Shopee Open Platform account once your APP application is approved.
* `shop_id` can be attained through this instruction: [Link](https://seller.shopee.com.my/edu/article/3805)
* `code` can be attained from your APP in your Shopee Open Platform account, through this instruction: [Link](https://open.shopee.com/developer-guide/20)

In [ ]:
# Predefined parameters
partner_id = 2229869 # for illustration purpose only, input your own `partner_id`
partner_key = "5c7a4e6868754d67594d6769566948754a4d74776359597578636c654a664a68" # for illustration purpose only, input your own `partner_key`
shop_id = 422221467 # for illustration purpose only, input your own `shop_id`
code = "4a4d4b4346496671656f536e496f626c" # for illustration purpose only, input your own `code`

# Function to retrieve access token
def get_token_shop_level(code, partner_id, partner_key, shop_id):
    timest = int(time.time())
    host = "https://partner.shopeemobile.com"
    path = "/api/v2/auth/token/get"
    body = {"code": code, "shop_id": shop_id, "partner_id": partner_id}
    
    # Generate signature
    tmp_base_string = "%s%s%s" % (partner_id, path, timest)
    base_string = tmp_base_string.encode()
    partner_key = partner_key.encode()
    sign = hmac.new(partner_key, base_string, hashlib.sha256).hexdigest()
    print("Auth Token Request Signature:", sign)
    
    # Construct URL
    url = f"{host}{path}?partner_id={partner_id}&timestamp={timest}&sign={sign}"
    print("Auth Token Request URL:", url)
    print("Auth Token Request Body:", json.dumps(body, indent=4))
    
    headers = {"Content-Type": "application/json"}
    resp = requests.post(url, json=body, headers=headers)
    print("Auth Token Response:", resp.text)
    
    ret = resp.json()
    access_token = ret.get("access_token")
    refresh_token = ret.get("refresh_token")
    return access_token, refresh_token

# Get access token
access_token, refresh_token = get_token_shop_level(code, partner_id, partner_key, shop_id)
print("Access Token:", access_token)
print("Refresh Token:", refresh_token)

Once `access_token` has been acquired and defined within the environment, we can make API calls to get data.

### get_item_extra_info
* Deck: [Link](https://open.shopee.com/documents/v2/v2.product.get_item_extra_info?module=89&type=1)

Since this API limits data retrieval to 50 SKUs per call, we will create a mother list of all our SKUs, divided into sublists, each containing 50 SKUs.

In [14]:
### sku to extract data
mother_sku_list = [[23061956391,
  17080951488,
  9952342927,
  13119554022,
  10568187216,
  10868192504,
  8452337712,
  13348395624,
  23250664798,
  20182692618,
  15197931698,
  18091706749,
  11202348515,
  3988319582,
  13246009344,
  8652313405,
  20043721993,
  7188244655,
  6288244851,
  15865125186,
  15618691283,
  5088478051,
  8452343592,
  21474383444,
  15490627861,
  18503301358,
  20503299499,
  15500215089,
  20813880703,
  15680165920,
  20771060979,
  20471064931,
  23264198705,
  18583263610,
  11594471132,
  14867211186,
  14018688016,
  14400183392,
  8596854711,
  10903888826,
  13846692938,
  8552315954,
  14665134321,
  12651682939,
  15300173415,
  9352331173,
  17132859172,
  23781573151,
  19603302164,
  8496629533],
 [18122713026,
  21732312747,
  11370393236,
  21155899616,
  22349810686,
  11870403231,
  22362204053,
  22187664281,
  21382736254,
  22150796991,
  24428038110,
  13683055688,
  12248183780,
  23958533722,
  11670373829,
  15384732423,
  14721468750,
  13894353413,
  10269938769,
  13646747499,
  24054601719,
  14496123420,
  19255894847,
  8152344291,
  25428108742,
  15765129329,
  12448372827,
  10070408774,
  19574359568,
  21674368255,
  22427225364,
  20071067774,
  9352310017,
  18021081420,
  23287261938,
  18284627319,
  12747686853,
  19652023852,
  20321078084,
  20548051533,
  21748046495,
  10970411421,
  19948050534,
  24980934229,
  29000294903,
  28250294970,
  27050305759,
  18493567037,
  15780135829,
  25020933976],
 [10702330659,
  26650294525,
  24402113442,
  14765148367,
  13148367614,
  3388329772,
  19593623012,
  13347623517,
  14865121943,
  7388837032,
  16493038461,
  18622725266,
  13347682660,
  21319354803,
  19719362626,
  11702334053,
  27955238535,
  27755241789,
  29205241595,
  24083454411,
  25251806241,
  16579443076,
  28055739844,
  16039852585,
  12847653127,
  27857561316,
  27207561509,
  24734608572,
  18922708683,
  28458866117,
  8555181359,
  27809235033,
  27109235578,
  25085664955,
  26461015897]]

### Retrieve data into a dataframe
1. Create helper functions to call the API correctly
2. Initialize an empty DataFrame to append data into
3. Make multiple API calls by looping through the sublists in mother list of SKUs
4. Export data into excel

In [ ]:
# Function to generate HMAC-SHA256 signature
def generate_sign(timest, partner_id, partner_key, shop_id, access_token):
    # API path for this request
    path = "/api/v2/product/get_item_extra_info"

    # Concatenate parameters to create the base string
    base_string = f"{partner_id}{path}{timest}{access_token}{shop_id}"
    base_string_encoded = base_string.encode()
    partner_key_encoded = partner_key.encode()

    # Generate the HMAC-SHA256 signature
    sign = hmac.new(partner_key_encoded, base_string_encoded, hashlib.sha256).hexdigest()

    # Print and return the signature
    print("Generated Signature:", sign)
    return sign

# Function to retrieve item extra info
def get_item_extra_info(item_id_list, access_token, partner_id, partner_key, shop_id):
    # Define endpoint details
    host = "https://partner.shopeemobile.com"
    path = "/api/v2/product/get_item_extra_info"

    # Calculate timestamp and signature
    timest = int(time.time())
    sign = generate_sign(timest, partner_id, partner_key, shop_id, access_token)

    # Construct the request URL
    url = (
        f"{host}{path}?"
        f"partner_id={partner_id}&timestamp={timest}&sign={sign}&shop_id={shop_id}&"
        f"access_token={access_token}&item_id_list={item_id_list}"
    )

    headers = {"Content-Type": "application/json"}
    response = requests.get(url, headers=headers)

    # Return the response
    return response.json()

# Initialize an empty DataFrame to append all data
item_data_global = pd.DataFrame()

# Loop through each sublist in mother_sku_list
for sublist in mother_sku_list:
    item_id_list = ",".join(map(str, sublist))

    # Retrieve item extra info
    item_info = get_item_extra_info(item_id_list, access_token, partner_id, partner_key, shop_id)
    response_data = json.dumps(item_info, indent=4)

    # Extracting the `item_list`
    parsed_json = json.loads(response_data)
    item_list = parsed_json["response"]["item_list"]

    # Converting to a Pandas DataFrame
    df = pd.DataFrame(item_list)

    # Append the processed data into the master DataFrame
    item_data_global = pd.concat([item_data_global, df], ignore_index=True)

# Export data
item_data_global.to_csv('item_data_global.csv')

Generated Signature: 90481c5b8b0736d10b1e0fec752ffe853195c6b0c57356bd71e7e4535b037b42
Generated Signature: e51d9622d6d6bfefdfe229f8b7d940cd411b962dac4dc3cff76a361b30efb4fc
Generated Signature: 4590f50d7c5e3129852a88006f24cd65dddc87c18a64cf26eae0807317a534ca


### Preview data

In [ ]:
item_data_global.head()

,item_id,sale,views,likes,rating_star,comment_count
0,23061956391,161,0,1,4.878788,33
1,17080951488,0,21,0,0.000000,0
2,9952342927,6,14,0,5.000000,2
3,13119554022,350,568,25,4.983607,53
4,10568187216,90,170,6,5.000000,18
